# Input own cover image to predict range of views

In [2]:
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np

input image into folder

In [4]:
import matplotlib.pyplot as plt

from PIL import Image
import os
import torch
from torchvision import datasets, transforms
import torchvision.models as models
img_height,img_width=(224,224)
batch_size = 32
t_img_transform={
    'test_img': transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

tdata_dir = './coverimage'
timage_datasets = {x: datasets.ImageFolder(os.path.join(tdata_dir, x),t_img_transform[x]) for x in ['test_img']}
tdataloaders    = {x: torch.utils.data.DataLoader(timage_datasets[x], batch_size=1, shuffle=True, num_workers=4) for x in ['test_img']}

tdataset_sizes  = {x: len(timage_datasets[x]) for x in ['test_img']}
tclass_names    = timage_datasets['test_img'].classes

print(tdataloaders)
for i in tdataloaders:
    print(i)
in_data_dir = r"./coverimage/test_img"    
in_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,)

in_generator = in_datagen.flow_from_directory(in_data_dir,
                                                   target_size=(img_height,img_width),
                                                   batch_size=batch_size,
                                                   class_mode="categorical",
                                                   subset="training")#set as training data                                  

{'test_img': <torch.utils.data.dataloader.DataLoader object at 0x00000214AB54FA90>}
test_img
Found 1 images belonging to 7 classes.


output the predicted range of views

In [9]:
import pandas as pd
import tensorflow as tf

model = tf.keras.models.load_model("./coverimage/Saved_Model/ResNet50_covers2.h5")
tfilenames = in_generator.filenames
tnb_samples = len(in_generator)

ty_prob=[]
ty_act=[]
in_generator.reset()
for _ in range(tnb_samples):
    X_t,Y_t = in_generator.next()
    ty_prob.append(model.predict(X_t))
    ty_act.append(Y_t)
    
tpredicted_class = [list(in_generator.class_indices.keys())[i.argmax()]for i in ty_prob]
tactual_class = [list(in_generator.class_indices.keys())[i.argmax()] for i in ty_act]

print("predict class:",tpredicted_class)

predict class: ['less_than_100']
